In [ ]:
# server.py
import asyncio
from typing import Set, Optional

from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from fastapi.responses import HTMLResponse, PlainTextResponse
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], 
    allow_credentials=True,
    allow_methods=["*"], 
    allow_headers=["*"],
    expose_headers=["*"],
)

viewer_sockets: Set[WebSocket] = set()
viewer_lock = asyncio.Lock()
last_frame: Optional[bytes] = None


INDEX_HTML = """<!doctype html>
<html lang="ko"><meta charset="utf-8">
<title>Realtime Phone Cam → Server</title>
<style>body{font:16px/1.4 system-ui,apple-system,Segoe UI,Roboto} a{display:block;margin:8px 0}</style>
<h1>📷 Realtime Phone Cam → Server</h1>
<p>• <a href="/sender">Sender(폰/PC 카메라 캡처)</a>
• <a href="/viewer">Viewer(PC 모니터링)</a></p>
<p>같은 서버에서 열면 자동 연결됩니다.</p>
<p><strong>LLM Service:</strong> 다른 서비스에서 wss://localhost:8443/ws/viewer 로 연결 가능</p>
</html>"""

SENDER_HTML = """<!doctype html>
<html lang="ko"><meta charset="utf-8">
<title>Sender - Front Camera</title>
<style>
body{font:16px/1.4 system-ui,apple-system,Segoe UI,Roboto;margin:16px}
#wrap{display:flex;gap:16px;flex-wrap:wrap;align-items:flex-start}
video,canvas,img{max-width: 45vw;width:360px;border-radius:12px;box-shadow:0 2px 10px #0001}
label{display:inline-block;margin:8px 0}
.controls{display:flex;gap:12px;flex-wrap:wrap;align-items:center}
.badge{display:inline-block;padding:3px 8px;border-radius:999px;background:#eee}
</style>
<h1>📤 Sender (앞 카메라 → 서버)</h1>

<div class="controls">
  <button id="startBtn">시작</button>
  <button id="stopBtn" disabled>중지</button>
  <label>FPS:
    <input id="fps" type="number" min="1" max="30" value="10" style="width:5em">
  </label>
  <label>가로폭(px):
    <input id="width" type="number" min="160" max="1280" value="640" style="width:6em">
  </label>
  <label>JPEG 품질(0.5~0.95):
    <input id="q" type="number" step="0.05" min="0.5" max="0.95" value="0.75" style="width:5em">
  </label>
  <span class="badge" id="status">DISCONNECTED</span>
</div>

<div id="wrap">
  <div>
    <h3>Local Preview</h3>
    <video id="video" autoplay playsinline muted></video>
  </div>
  <div>
    <h3>전송 중 프레임</h3>
    <canvas id="canvas"></canvas>
  </div>
</div>

<script>
const startBtn = document.getElementById('startBtn');
const stopBtn  = document.getElementById('stopBtn');
const statusEl = document.getElementById('status');
const video = document.getElementById('video');
const canvas = document.getElementById('canvas');

let ws, stream, timer;
let running = false;

function wsUrl(path) {
  const proto = location.protocol === 'https:' ? 'wss' : 'ws';
  return `${proto}://${location.host}${path}`;
}

async function start() {
  if (running) return;
  running = true;
  startBtn.disabled = true;
  stopBtn.disabled  = false;

  const fps     = parseInt(document.getElementById('fps').value || '10', 10);
  const width   = parseInt(document.getElementById('width').value || '640', 10);
  const quality = Math.max(0.5, Math.min(0.95, parseFloat(document.getElementById('q').value || '0.75')));

  try {
    // 앞(셀피) 카메라 우선
    stream = await navigator.mediaDevices.getUserMedia({
      video: {
        facingMode: "user",
        width: { ideal: width },
        height: { ideal: Math.round(width * 3 / 4) }
      },
      audio: false
    });
  } catch (e) {
    alert('카메라 접근 실패: ' + e);
    running = false; startBtn.disabled = false; stopBtn.disabled = true;
    return;
  }

  video.srcObject = stream;
  await video.play();

  const w = width;
  const h = Math.round(width * (video.videoHeight / video.videoWidth || 0.75));
  canvas.width = w;
  canvas.height = h;
  const ctx = canvas.getContext('2d', { alpha: false, desynchronized: true });

  ws = new WebSocket(wsUrl('/ws/sender'));
  ws.binaryType = 'arraybuffer';
  ws.onopen = () => statusEl.textContent = 'CONNECTED';
  ws.onclose = () => statusEl.textContent = 'DISCONNECTED';
  ws.onerror = () => statusEl.textContent = 'ERROR';

  let sending = false;

  const sendFrame = async () => {
    if (!running || ws.readyState !== 1) return;
    if (sending) return; // backpressure
    sending = true;

    ctx.drawImage(video, 0, 0, w, h);
    canvas.toBlob(async (blob) => {
      try {
        if (blob) {
          const buf = await blob.arrayBuffer();
          ws.send(buf);
        }
      } catch(_){}
      sending = false;
    }, 'image/jpeg', quality);
  };

  timer = setInterval(sendFrame, Math.max(1000 / fps, 20));

  // 탭 전환 시 전송 일시정지/재개
  document.addEventListener('visibilitychange', () => {
    if (document.hidden) {
      if (timer) { clearInterval(timer); timer = null; }
    } else {
      if (!timer) timer = setInterval(sendFrame, Math.max(1000 / fps, 20));
    }
  });
}

function stop() {
  running = false;
  startBtn.disabled = false;
  stopBtn.disabled = true;

  if (timer) { clearInterval(timer); timer = null; }
  if (ws && ws.readyState === 1) ws.close();
  if (stream) {
    for (const t of stream.getTracks()) t.stop();
    stream = null;
  }
  statusEl.textContent = 'DISCONNECTED';
}

startBtn.onclick = start;
stopBtn.onclick  = stop;
</script>
</html>
"""

VIEWER_HTML = """<!doctype html>
<html lang="ko"><meta charset="utf-8">
<title>Viewer - Monitor</title>
<style>
body{font:16px/1.4 system-ui,apple-system,Segoe UI,Roboto;margin:16px}
img{max-width:80vw;border-radius:12px;box-shadow:0 2px 10px #0001}
.badge{display:inline-block;padding:3px 8px;border-radius:999px;background:#eee}
</style>
<h1>👀 Viewer (수신 영상)</h1>
<p><span class="badge" id="status">DISCONNECTED</span></p>
<img id="view" alt="stream will appear here"/>
<script>
const statusEl = document.getElementById('status');
const img = document.getElementById('view');

function wsUrl(path) {
  const proto = location.protocol === 'https:' ? 'wss' : 'ws';
  return `${proto}://${location.host}${path}`;
}

let ws;
function connect() {
  ws = new WebSocket(wsUrl('/ws/viewer'));
  ws.binaryType = 'blob';
  ws.onopen = () => statusEl.textContent = 'CONNECTED';
  ws.onclose = () => { statusEl.textContent = 'DISCONNECTED'; setTimeout(connect, 1500); };
  ws.onerror = () => statusEl.textContent = 'ERROR';
  ws.onmessage = (ev) => {
    const url = URL.createObjectURL(ev.data);
    img.onload = () => URL.revokeObjectURL(url);
    img.src = url;
  };
}
connect();
</script>
</html>
"""

@app.get("/", response_class=HTMLResponse)
async def index():
    return INDEX_HTML

@app.get("/sender", response_class=HTMLResponse)
async def sender_page():
    return SENDER_HTML

@app.get("/viewer", response_class=HTMLResponse)
async def viewer_page():
    return VIEWER_HTML

@app.get("/healthz", response_class=PlainTextResponse)
async def health():
    return "ok"

@app.websocket("/ws/sender")
async def ws_sender(ws: WebSocket):
    global last_frame
    await ws.accept()
    try:
        while True:
            data = await ws.receive_bytes()
            last_frame = data
            # broadcast to viewers
            stale = []
            async with viewer_lock:
                for v in list(viewer_sockets):
                    try:
                        await v.send_bytes(data)
                    except Exception:
                        stale.append(v)
                for v in stale:
                    viewer_sockets.discard(v)
    except WebSocketDisconnect:
        pass
    except Exception:
        pass

@app.websocket("/ws/viewer")
async def ws_viewer(ws: WebSocket):
    await ws.accept()
    async with viewer_lock:
        viewer_sockets.add(ws)
    # 최초 프레임 있으면 즉시 전송
    if last_frame:
        try:
            await ws.send_bytes(last_frame)
        except Exception:
            pass
    try:
        # keep alive (전송은 sender에서 push)
        while True:
            await asyncio.sleep(60)
    except WebSocketDisconnect:
        pass
    finally:
        async with viewer_lock:
            viewer_sockets.discard(ws)



In [4]:
import nest_asyncio, uvicorn
nest_asyncio.apply()

#from server import app  # server.py에 정의된 FastAPI app 객체
# uvicorn.run(app, host="0.0.0.0", port=8000, reload=False)


uvicorn.run(
    app,
    host="0.0.0.0",
    port=8443,
    ssl_keyfile="./server.key",
    ssl_certfile="./server.crt",
    reload=False,
)

INFO:     Started server process [72358]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on https://0.0.0.0:8443 (Press CTRL+C to quit)


INFO:     192.168.99.247:52707 - "GET / HTTP/1.1" 200 OK
INFO:     192.168.99.247:52707 - "GET /sender HTTP/1.1" 200 OK
INFO:     192.168.99.247:52709 - "GET /sender HTTP/1.1" 200 OK
INFO:     192.168.99.16:58433 - "GET / HTTP/1.1" 200 OK
INFO:     192.168.99.16:58433 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     192.168.99.16:58433 - "GET /sender HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [72358]
